# Mistral and Weights & Biases: Finetune an LLM judge so detect hallucination

In this notebooks you will learn how to trace your MistralAI Api calls using W&B Weave, how to evaluate the performance of your models and how to close the gap by leveraging the MistralAI finetuning capabilities.

In this notebooks we will fine-tune a mistral 7b model as an LLM Judge. This idea comes from the [amazing blog post from Eugene](https://eugeneyan.com/writing/finetuning/). The main goal is to fine-tune a small model like Mistral 7B to act as an hallucination judge. We will do this in 2 steps:
- Training on a [Factual Inconsistency Benchmark](https://arxiv.org/abs/2211.08412v1) challenging dataset to improve the model performance to detect hallucination by detecting inconsistencies beween a piece of text and a "summary"
- We will then mix that dataset with Wikipedia summaries dataset to increase the performance even more.

[![](static/eugene1.png)](https://eugeneyan.com/writing/finetuning/)


---


- Weights & Biases: https://wandb.ai/
- Mistral finetuning docs: https://docs.mistral.ai/capabilities/finetuning/
- Tracing with W&B Weave: https://wandb.me/weave

## Load some data

In [1]:
# !pip install mistralai pandas weave

let's import the relevant pieces

In [2]:
import os, asyncio, json
from pathlib import Path

import weave

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

client = MistralClient(api_key=os.environ["MISTRAL_API_KEY"])

some globals

In [4]:
DATA_PATH = Path("./data")
NUM_SAMPLES = 100 # Number of samples to use for evaluation, use None for all
PROJECT_NAME = "llm-judge-webinar"

In [5]:
weave.init(PROJECT_NAME)

Logged in as Weights & Biases user: capecape.
View Weave data at https://wandb.ai/capecape/llm-judge-webinar/weave


In [6]:
def read_jsonl(path):
    "returns a list of dictionaries"
    with open(path, 'r') as file:
        return [json.loads(line) for line in file]

In [7]:
train_ds = read_jsonl(DATA_PATH / "fib-train.jsonl")
val_ds = read_jsonl(DATA_PATH / "fib-val.jsonl")[0:NUM_SAMPLES]

![](static/nli.png)

We are going to map to 0 and 1 for the sake of it!

In [8]:
len(train_ds)

820

In [9]:
train_ds[0]

{'premise': 'Vehicles and pedestrians will now embark and disembark the Cowes ferry separately following Maritime and Coastguard Agency (MCA) guidance.\nIsle of Wight Council said its new procedures were in response to a resident\'s complaint.\nCouncillor Shirley Smart said it would "initially result in a slower service".\nOriginally passengers and vehicles boarded or disembarked the so called "floating bridge" at the same time.\nMs Smart, who is the executive member for economy and tourism, said the council already had measures in place to control how passengers and vehicles left or embarked the chain ferry "in a safe manner".\nHowever, it was "responding" to the MCA\'s recommendations "following this complaint".\nShe added: "This may initially result in a slower service while the measures are introduced and our customers get used to the changes."\nThe service has been in operation since 1859.',
 'hypothesis': "A new service on the Isle of Wight's chain ferry has been launched followi

You will probably integrate MistralAI API calls in your codebase by creating a function like the one below:

In [10]:
@weave.op()  # <---- add this and you are good to go
def call_mistral(model:str, messages:list, **kwargs) -> str:
    "Call the Mistral API"
    chat_response = client.chat(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
        **kwargs,
    )
    return json.loads(chat_response.choices[0].message.content)

Let's create a prompt that explains the task...

In [13]:
prompt = """You are an expert to detect factual inconsistencies and hallucinations. You will be given a document and a summary.
- Carefully read the full document and the provided summary.
- Identify Factual Inconsistencies: any statements in the summary that are not supported by or contradict the information in the document.
Factually Inconsistent: If any statement in the summary is not supported by or contradicts the document, label it as 0
Factually Consistent: If all statements in the summary are supported by the document, label it as 1

Highlight or list the specific statements in the summary that are inconsistent.
Provide a brief explanation of why each highlighted statement is inconsistent with the document.

Return in JSON format with `consistency` and a `reason` for the given choice.

Document: 
{premise}
Summary: 
{hypothesis}
"""

In [14]:
def format_prompt(prompt, premise: str, hypothesis: str, cls=ChatMessage):
    messages = [
        cls(
            role="user", 
            content=prompt.format(premise=premise, hypothesis=hypothesis)
        )
    ]
    return messages

In [15]:
premise, hypothesis, target = train_ds[1]['premise'], train_ds[1]['hypothesis'], train_ds[1]['target']
messages=format_prompt(prompt, premise, hypothesis)
output = call_mistral(model="open-mistral-7b", messages=messages)

🍩 https://wandb.ai/capecape/llm-judge-webinar/r/call/65f0cab2-162c-4f1f-b027-c1e361955edf


In [16]:
output

{'consistency': 1,
 'reason': 'The summary is factually consistent with the document. Both mention the new safety measures implemented on the Cowes ferry, which will initially result in a longer crossing time. The document also mentions the complaint from a resident and the response from the Isle of Wight Council, but these details are not contradictory to the longer crossing times.'}

In [17]:
target

1

In [18]:
class MistralModel(weave.Model):
    model: str
    prompt: str
    temperature: float = 0.7
    
    @weave.op
    def create_messages(self, premise:str, hypothesis:str):
        return format_prompt(self.prompt, premise, hypothesis)

    @weave.op
    def predict(self, premise:str, hypothesis:str):
        messages = self.create_messages(premise, hypothesis)
        return call_mistral(model=self.model, messages=messages, temperature=self.temperature)

In [19]:
model_7b = MistralModel(model="open-mistral-7b", prompt=prompt, temperature=0.7)
output = model_7b.predict(premise, hypothesis)

🍩 https://wandb.ai/capecape/llm-judge-webinar/r/call/736605d5-5c2f-43c0-9801-047cd8811127


## Eval

Let's evaluate the model on the validation split of the dataset

In [20]:
def accuracy(model_output, target):
    class_model_output = model_output.get('consistency') if model_output else None
    return {"accuracy": class_model_output == target}

In [21]:
class BinaryMetrics(weave.Scorer):
    class_name: str
    eps: float = 1e-8

    @weave.op()
    def summarize(self, score_rows) -> dict:
        # filter out None rows, model may error out sometimes...
        score_rows = [score for score in score_rows if score["correct"] is not None]
        # Compute f1, precision, recall
        tp = sum([not score["negative"] and score["correct"] for score in score_rows])
        fp = sum([not score["negative"] and not score["correct"] for score in score_rows])
        fn = sum([score["negative"] and not score["correct"] for score in score_rows])
        precision = tp / (tp + fp + self.eps)
        recall = tp / (tp + fn + self.eps)
        f1 = 2 * precision * recall / (precision + recall + self.eps)
        result = {"f1": f1, "precision": precision, "recall": recall}
        return result

    @weave.op()
    def score(self, target: dict, model_output: dict) -> dict:
        class_model_output = model_output.get(self.class_name) if model_output else None  # 0 or 1
        result = {
            "correct": class_model_output == target,
            "negative": not class_model_output,
        }
        return result

F1 = BinaryMetrics(class_name="consistency")

In [22]:
evaluation = weave.Evaluation(dataset=val_ds, scorers=[accuracy, F1])

## 7B

In [23]:
await evaluation.evaluate(model_7b)

Evaluated 1 of 100 examples

Evaluated 2 of 100 examples

Evaluated 3 of 100 examples

Evaluated 4 of 100 examples

Evaluated 5 of 100 examples

...

Evaluated 99 of 100 examples

Evaluated 100 of 100 examples

Evaluation summary
{
    'model_output': {'consistency': {'mean': 0.41}},
    'accuracy': {'accuracy': {'true_count': 63, 'true_fraction': 0.63}},
    'BinaryMetrics': {'f1': 0.5934065883250815, 'precision': 0.658536585205235, 'recall': 0.539999999892},
    'model_latency': {'mean': 3.4978641939163206}
}

🍩 https://wandb.ai/capecape/llm-judge-webinar/r/call/7c45007e-c48f-453c-959e-408aa854a177


{'model_output': {'consistency': {'mean': 0.41}},
 'accuracy': {'accuracy': {'true_count': 63, 'true_fraction': 0.63}},
 'BinaryMetrics': {'f1': 0.5934065883250815,
  'precision': 0.658536585205235,
  'recall': 0.539999999892},
 'model_latency': {'mean': 3.4978641939163206}}

![](static/eval_7b.png)

## Iterate a bit on the prompt...

Let's try adding the example from Eugene's blog:

In [27]:
prompt_example = """You are an expert to detect factual inconsistencies and hallucinations. You will be given a document and a summary.
- Carefully read the full document and the provided summary.
- Identify Factual Inconsistencies: any statements in the summary that are not supported by or contradict the information in the document.
Factually Inconsistent: If any statement in the summary is not supported by or contradicts the document, label it as 0
Factually Consistent: If all statements in the summary are supported by the document, label it as 1

Here you have an example:

Document: 
Vehicles and pedestrians will now embark and disembark the Cowes ferry separately following Maritime and Coastguard Agency (MCA) guidance. 
Isle of Wight Council said its new procedures were in response to a resident’s complaint. Councillor Shirley Smart said it would 
“initially result in a slower service”. Originally passengers and vehicles boarded or disembarked the so-called “floating bridge” at the same time. 
Ms Smart, who is the executive member for economy and tourism, said the council already had measures in place to control how passengers 
and vehicles left or embarked the chain ferry “in a safe manner”. However, it was “responding” to the MCA’s recommendations “following this 
complaint”. She added: “This may initially result in a slower service while the measures are introduced and our customers get used to 
the changes.” The service has been in operation since 1859.

Inconsistent summary: A new service on the Isle of Wight’s chain ferry has been launched following a complaint from a resident.

Consistent summary: Passengers using a chain ferry have been warned crossing times will be longer because of new safety measures.

Highlight or list the specific statements in the summary that are inconsistent.
Provide a brief explanation of why each highlighted statement is inconsistent with the document.

Return in JSON format with `consistency` and a `reason` for the given choice.

Document: 
{premise}
Summary: 
{hypothesis}
"""

In [28]:
model_7b_ex = MistralModel(model="open-mistral-7b", prompt=prompt_example, temperature=0.7)
output = model_7b_ex.predict(premise, hypothesis)

🍩 https://wandb.ai/capecape/llm-judge-webinar/r/call/1bf0b0ef-0b2e-4610-b83f-4eb725270065


In [29]:
output

{'consistency': 1,
 'reason': "The summary is factually consistent with the document. It correctly states that the new safety measures on the chain ferry will result in longer crossing times, which is supported by the statement 'Ms Smart, who is the executive member for economy and tourism, said the council already had measures in place to control how passengers and vehicles left or embarked the chain ferry "}

This is a hard dataset!

In [30]:
await evaluation.evaluate(model_7b_ex)

Evaluated 1 of 100 examples

Evaluated 2 of 100 examples

Evaluated 3 of 100 examples

Evaluated 4 of 100 examples

Evaluated 5 of 100 examples

...

Evaluated 99 of 100 examples

Evaluated 100 of 100 examples

Evaluation summary
{
    'model_output': {'consistency': {'mean': 0.22}},
    'accuracy': {'accuracy': {'true_count': 56, 'true_fraction': 0.56}},
    'BinaryMetrics': {'f1': 0.388888884537037, 'precision': 0.6363636360743802, 'recall': 0.279999999944},
    'model_latency': {'mean': 3.6455528283119203}
}

🍩 https://wandb.ai/capecape/llm-judge-webinar/r/call/2bed90db-93cf-4cb2-8fd3-999bdd0600e5


{'model_output': {'consistency': {'mean': 0.22}},
 'accuracy': {'accuracy': {'true_count': 56, 'true_fraction': 0.56}},
 'BinaryMetrics': {'f1': 0.388888884537037,
  'precision': 0.6363636360743802,
  'recall': 0.279999999944},
 'model_latency': {'mean': 3.6455528283119203}}

## Large

In [31]:
model_large = MistralModel(model="mistral-large-latest", prompt=prompt_example, temperature=0.7)
await evaluation.evaluate(model_large)

Evaluated 1 of 100 examples

Evaluated 2 of 100 examples

Evaluated 3 of 100 examples

Evaluated 4 of 100 examples

Evaluated 5 of 100 examples

...

Evaluated 99 of 100 examples

Evaluated 100 of 100 examples

Evaluation summary
{
    'model_output': {'consistency': {'mean': 0.5}},
    'accuracy': {'accuracy': {'true_count': 78, 'true_fraction': 0.78}},
    'BinaryMetrics': {'f1': 0.7799999948440001, 'precision': 0.779999999844, 'recall': 0.779999999844},
    'model_latency': {'mean': 4.479781422615051}
}

🍩 https://wandb.ai/capecape/llm-judge-webinar/r/call/48a173f9-ceff-46f7-b213-a1462706c966


{'model_output': {'consistency': {'mean': 0.5}},
 'accuracy': {'accuracy': {'true_count': 78, 'true_fraction': 0.78}},
 'BinaryMetrics': {'f1': 0.7799999948440001,
  'precision': 0.779999999844,
  'recall': 0.779999999844},
 'model_latency': {'mean': 4.479781422615051}}

![](static/eval_large.png)

This model is considerably better! over 80% accuracy is great on this hard task 😎

## Fine-Tune FTW

Let's see if fine-tuning improves this.

In [32]:
ft_prompt = """You are an expert to detect factual inconsistencies and hallucinations. You will be given a document and a summary.
- Carefully read the full document and the provided summary.
- Identify Factual Inconsistencies: any statements in the summary that are not supported by or contradict the information in the document.
Factually Inconsistent: If any statement in the summary is not supported by or contradicts the document, label it as 0
Factually Consistent: If all statements in the summary are supported by the document, label it as 1

Return in JSON format with `consistency` for the given choice.

Document: 
{premise}
Summary: 
{hypothesis}
"""

answer = """{{"consistency": {label}}}"""  # <- json schema

You will need to format your prompts slightly different for FT
- instead of `ChatMessage` use a `dict`
- Add the output

In [33]:
def format_prompt_ft(row, cls=dict, with_answer=True):
    "Format on the expected MistralAI fine-tuning dataset"
    premise = row['premise']
    hypothesis = row['hypothesis']
    messages = [
        cls(
            role="user", 
            content=prompt.format(premise=premise, hypothesis=hypothesis)
        )
    ]
    if with_answer:
        label = row['target']
        messages.append(
            cls(
                role="assistant",
            content=answer.format(label=label)
            )
        )
    return messages

In [34]:
format_prompt_ft(train_ds[0])

[{'role': 'user',
  'content': 'You are an expert to detect factual inconsistencies and hallucinations. You will be given a document and a summary.\n- Carefully read the full document and the provided summary.\n- Identify Factual Inconsistencies: any statements in the summary that are not supported by or contradict the information in the document.\nFactually Inconsistent: If any statement in the summary is not supported by or contradicts the document, label it as 0\nFactually Consistent: If all statements in the summary are supported by the document, label it as 1\n\nHighlight or list the specific statements in the summary that are inconsistent.\nProvide a brief explanation of why each highlighted statement is inconsistent with the document.\n\nReturn in JSON format with `consistency` and a `reason` for the given choice.\n\nDocument: \nVehicles and pedestrians will now embark and disembark the Cowes ferry separately following Maritime and Coastguard Agency (MCA) guidance.\nIsle of Wigh

You could use other fancy datasets or pandas, but this is a small dataset so let's not add more complexity...

In [35]:
formatted_train_ds = [format_prompt_ft(row) for row in train_ds]
formatted_val_ds = [format_prompt_ft(row) for row in val_ds]

In [36]:
def save_jsonl(ds, path):
    with open(path, "w") as f:
        for row in ds:
            f.write(json.dumps(row) + "\n")
save_jsonl(formatted_train_ds, DATA_PATH/"formatted_fib_train.jsonl")
save_jsonl(formatted_val_ds, DATA_PATH/"formatted_fib_val.jsonl")

## Upload dataset

In [37]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open(DATA_PATH/"formatted_fib_train.jsonl", "rb") as f:
    ds_train = client.files.create(file=("formatted_df_train.jsonl", f))
with open(DATA_PATH/"formatted_fib_val.jsonl", "rb") as f:
    ds_eval = client.files.create(file=("eval.jsonl", f))

In [38]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [39]:
pprint(ds_train)

{
    "id": "1a3ccdb9-4b8b-49f7-9c4b-ee60b8043a88",
    "object": "file",
    "bytes": 2032975,
    "created_at": 1720179083,
    "filename": "formatted_df_train.jsonl",
    "purpose": "fine-tune"
}


## Create a fine-tuning job

Ok, now let's create a fine-tune job with the mistral api. Some thing to know:
- You only have 2 parameters to play wtih: `training_steps` and `learning_rate`
- You can use `dry_run=True` to get an estimate cost
- `training_steps` is not exactly linked to epochs in a direct way, they have a rule of thumbs on the docs. If you do a dry run the epochs will be calculated for you.

We want to run for 10 epochs to reproduce Eugene's results.

In [42]:
from mistralai.models.jobs import TrainingParameters, WandbIntegrationIn

created_jobs = client.jobs.create(
    # dry_run=True,
    model="open-mistral-7b",
    training_files=[ds_train.id],
    validation_files=[ds_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=35,
        learning_rate=0.0001,
        ),
    integrations=[
        WandbIntegrationIn(
            project=PROJECT_NAME,
            run_name="mistral_7b_fib",
            api_key=os.environ.get("WANDB_API_KEY"),
        ).dict()
    ],
)

In [43]:
pprint(created_jobs)

{
    "id": "f3051e44-4b86-4a78-875a-dfccbc4cdf38",
    "hyperparameters": {
        "training_steps": 35,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1720179097,
    "modified_at": 1720179097,
    "training_files": [
        "1a3ccdb9-4b8b-49f7-9c4b-ee60b8043a88"
    ],
    "validation_files": [
        "6be77e5f-91fd-461f-8d73-22615f242b79"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "llm-judge-webinar",
            "name": null,
            "run_name": "mistral_7b_fib"
        }
    ]
}


In [ ]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)

![](static/ft_dashboard.png)

## Use a fine-tuned model

Let's compute the predictions using the fine-tuned 7B model

In [45]:
jobs = client.jobs.list()

In [46]:
retrieved_job = jobs.data[0]

In [47]:
retrieved_job.fine_tuned_model

'ft:open-mistral-7b:0362203c:20240705:f3051e44'

In [48]:
mistral_7b_ft = MistralModel(prompt=ft_prompt, model=retrieved_job.fine_tuned_model)

In [49]:
await evaluation.evaluate(mistral_7b_ft)

Evaluated 1 of 100 examples

Evaluated 2 of 100 examples

Evaluated 3 of 100 examples

Evaluated 4 of 100 examples

Evaluated 5 of 100 examples

...

Evaluated 99 of 100 examples

Evaluated 100 of 100 examples

Evaluation summary
{
    'model_output': {'consistency': {'mean': 0.57}},
    'accuracy': {'accuracy': {'true_count': 79, 'true_fraction': 0.79}},
    'BinaryMetrics': {'f1': 0.8037383126281773, 'precision': 0.7543859647799322, 'recall': 0.859999999828},
    'model_latency': {'mean': 2.8312768507003785}
}

🍩 https://wandb.ai/capecape/llm-judge-webinar/r/call/af3ef7a1-2f80-4786-bdce-988454f278ad


{'model_output': {'consistency': {'mean': 0.57}},
 'accuracy': {'accuracy': {'true_count': 79, 'true_fraction': 0.79}},
 'BinaryMetrics': {'f1': 0.8037383126281773,
  'precision': 0.7543859647799322,
  'recall': 0.859999999828},
 'model_latency': {'mean': 2.8312768507003785}}

quite substantial improvement! Some take aways:
- the Mistral 7B is a much more powerful model than the original Bart that eugene was using on his blog post
- With a relatively small high quality dataset the improvements for this downstream task are enormous!
- Now we can leverage a faster and cheaper 7B instead of taping into `mistral-large`. Of course we could have some filtering logic to decide when to use the big gun anyway.

## USB

Let's mix the USB dataset in the training data...

In [53]:
train_ds_usb = read_jsonl(DATA_PATH / "usb-train.jsonl")

formatted_train_usb_ds = [format_prompt_ft(row) for row in train_ds_usb]
save_jsonl(formatted_train_usb_ds, DATA_PATH/"formatted_train_usb.jsonl")

In [54]:
with open(DATA_PATH/"formatted_train_usb.jsonl", "rb") as f:
    ds_train_usb = client.files.create(file=("formatted_df_train_usb.jsonl", f))

In [62]:
from mistralai.models.jobs import TrainingParameters, WandbIntegrationIn

created_jobs = client.jobs.create(
    # dry_run=True,
    model="open-mistral-7b",
    training_files=[ds_train.id, ds_train_usb.id], # <- just add this new file
    validation_files=[ds_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=200,
        learning_rate=0.0001,
        ),
    integrations=[
        WandbIntegrationIn(
            project=PROJECT_NAME,
            run_name="mistral_7b_fib_usb", # <- change the run name
            api_key=os.environ.get("WANDB_API_KEY"),
        ).dict()
    ],
)

In [84]:
jobs = client.jobs.list()

In [86]:
created_jobs = jobs.data[1]

In [87]:
pprint(created_jobs)

{
    "id": "7687f4d8-cd23-4012-af6c-67c52ab56129",
    "hyperparameters": {
        "training_steps": 200,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "RUNNING",
    "job_type": "FT",
    "created_at": 1720180272,
    "modified_at": 1720180274,
    "training_files": [
        "951fa86e-e993-4c6c-a0b9-9d03ae067e16",
        "1a3ccdb9-4b8b-49f7-9c4b-ee60b8043a88"
    ],
    "validation_files": [
        "6be77e5f-91fd-461f-8d73-22615f242b79"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "llm-judge-webinar",
            "name": null,
            "run_name": "mistral_7b_fib_usb"
        }
    ]
}


In [88]:
retrieved_job = client.jobs.retrieve(created_jobs.id)

In [89]:
retrieved_job.fine_tuned_model

'ft:open-mistral-7b:0362203c:20240705:7687f4d8'

In [90]:
mistral_7b_usb_ft = MistralModel(prompt=ft_prompt, model=retrieved_job.fine_tuned_model)

In [91]:
await evaluation.evaluate(mistral_7b_usb_ft)

Evaluated 1 of 100 examples

Evaluated 2 of 100 examples

Evaluated 3 of 100 examples

Evaluated 4 of 100 examples

Evaluated 5 of 100 examples

...

Evaluated 99 of 100 examples

Evaluated 100 of 100 examples

Evaluation summary
{
    'model_output': {'consistency': {'mean': 0.54}},
    'accuracy': {'accuracy': {'true_count': 90, 'true_fraction': 0.9}},
    'BinaryMetrics': {'f1': 0.9038461486797338, 'precision': 0.8703703702091906, 'recall': 0.939999999812},
    'model_latency': {'mean': 2.8983353924751283}
}

🍩 https://wandb.ai/capecape/llm-judge-webinar/r/call/4e6f6a62-c592-40de-bfce-2ac7d19707e6


{'model_output': {'consistency': {'mean': 0.54}},
 'accuracy': {'accuracy': {'true_count': 90, 'true_fraction': 0.9}},
 'BinaryMetrics': {'f1': 0.9038461486797338,
  'precision': 0.8703703702091906,
  'recall': 0.939999999812},
 'model_latency': {'mean': 2.8983353924751283}}

## Final results

The fine-tuned model over USB + FIB is now 90%+ accurate!

![](static/eval_final.png)